# #2 Even-odd Sudoku - Formulation

_Author: Luiz Suzana_  
_July, 2021_

This is the MIP formulation of the puzzle. Statement and solution implementation of all puzzles 
are available from the main page of the [Fun Puzzles](https://mip-master.github.io/puzzles/) project, 
which is maintained by [Mip Master](https://mipmaster.org/).

## <a name="section-formulation"></a>Formulation

The goal is to fill a 9x9 grid with digits from 1 to 9 in a way that the following conditions are satisfied: 

- Usual rules of sudoku, i.e., in each row, column and bold region, all the digits from 1 to 9 must appear, or equivalently, there must be no repetition of any digit in each row, column and bold region;

- The given initial digits on the cells must be preserved at the final solution;

- Some specific cells must have an even/odd digit (see [Fun Puzzles](https://mip-master.github.io/puzzles/) project for the puzzle statement).


<img src="../../figures/2_even_odd_sudoku.png" alt="anything" width="300"/>

### Input Data
We start by defining the set (which is technically a list) of indices, which corresponds simultaneously to the available digits, and to the indices of rows and columns:
- `Index = [1, 2, 3, 4, 5, 6, 7, 8, 9]`

We also need to describe the cells where we are given: an even digit; an odd digit; an initial digit:

- Even cells:  
    `EC = [(2, 1), (3, 2), (3, 5), (2, 6), (2, 7), (2, 8), (3, 8), (4, 8), (5, 7), (8, 7), (8, 9)]`


- Odd cells:  
    `OC = [(1, 2), (2, 3), (4, 4), (5, 3), (6, 2), (6, 6), (7, 2), (8, 2), (8, 3), (8, 4), (7, 5), (7, 8), (9, 8)]`


- Given digits:  
    `GD = {(1, 6): 4, (1, 7): 6, (1, 9): 9, (2, 5): 5, (3, 4): 1, (3, 9): 7, (4, 3): 4, (4, 9): 8, (5, 2): 2, (5, 8): 9, (6, 1): 1, (6, 7): 3, (7, 1): 9, (7, 6): 8, (8, 5): 6, (9, 1): 8, (9, 3): 5, (9, 4): 7}`


We describe now the bold regions, as a python dictionary, ordering them in any way:  
- Bold regions:  
    ```
    BR = {
          1: [(1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3)],
          2: [(1, 4), (1, 5), (1, 6), (2, 4), (2, 5), (2, 6), (3, 4), (3, 5), (3, 6)],
          3: [(1, 7), (1, 8), (1, 9), (2, 7), (2, 8), (2, 9), (3, 7), (3, 8), (3, 9)],
          4: [(4, 1), (4, 2), (4, 3), (5, 1), (5, 2), (5, 3), (6, 1), (6, 2), (6, 3)],
          5: [(4, 4), (4, 5), (4, 6), (5, 4), (5, 5), (5, 6), (6, 4), (6, 5), (6, 6)],
          6: [(4, 7), (4, 8), (4, 9), (5, 7), (5, 8), (5, 9), (6, 7), (6, 8), (6, 9)],
          7: [(7, 1), (7, 2), (7, 3), (8, 1), (8, 2), (8, 3), (9, 1), (9, 2), (9, 3)],
          8: [(7, 4), (7, 5), (7, 6), (8, 4), (8, 5), (8, 6), (9, 4), (9, 5), (9, 6)],
          9: [(7, 7), (7, 8), (7, 9), (8, 7), (8, 8), (8, 9), (9, 7), (9, 8), (9, 9)]
          }
    ```
    
Finally, we define the list of indices for decision variables (see [Decision Variables](#section-decision-variables)):

- `keys = [(i, j, k) for i in Index for j in Index for k in Index]`

### <a name="section-decision-variables"></a>Decision Variables
We now define the decision variables. The first attempt could be to define a set of variables $x_{ij}\in\{1, \ldots, 9\}$ for $i, j \in \{1, \ldots, 9\}$ which correspond to the digit that must enter cell $(i, j)$. Although this formulation is possible, it gets hard to establish the constraints later (why? Try yourself).

Alternatively, we propose the following decision variables:

- $x_{ijk}\in\{0, 1\}$ for $i, j, k \in\{1, \ldots, 9\}$, where $x_{ijk} = 1$ when digit $k$ enters cell $(i, j)$, and $x_{ijk} = 0$ otherwise. 

At first glance, this may seem an over-complicated formulation, but it will save us a lot of effort while setting the constraints. Note that in this way, we have a bigger amount of variables, but they turn out to be binaries!

### Constraints
We saw a general description for the constraints in [Formulation](#section-formulation). With the decision variables set, we now establish the constraints precisely:

- _Each cell must have exactly one digit_: $$ \forall i, j\in\{1, \ldots, 9\}\;\; :\;\; \sum_k x_{ijk} = 1 $$
    
    Here, a pair $(i, j)$ corresponds to one cell in the 9x9 grid, and since the variables are binaries, the sum over $k$ above enforces one, and exactly one, of the $x_{ijk}$ to be $1$. Therefore, the unique $k$ for which $x_{ijk} = 1$ represents the digit that enters cell $(i, j)$, which guarantees the desired constraint. There would not be such constraint if we had defined the decision variables in the other way.


- _Digits can't repeat in each row_: $$ \forall i,k \in \{1, \ldots, 9\}\;\; : \;\; \sum_j x_{ijk} = 1 $$
    
    $i$ corresponds to each row one the grid, while $k$ stands for each avaiable digit. Then, given a row $i$ and a digit $k$, the sum over $j$ guarantees the existence of one, and exactly one, $j$ such that $x_{ijk} = 1$, i.e., just one $j$ for which the value $k$ enters $(i, j)$.


- _Digits can't repeat in each column_: $$ \forall j,k \in \{1, \ldots, 9\}\;\; : \;\; \sum_i x_{ijk} = 1 $$
    
    $j$ represents each column, and $k$ each avaiable digit. It is similar to the previous constraint.


- _Digits can't repeat in each bold region_: $$ \forall b \in BR, \;\forall k \in\{1, \ldots, 9\} \;\; : \;\; \sum_{(i,j)\in b} x_{ijk} = 1 $$

    Now, for a given bold region $b\in BR$ and a given digit $k$, the sum over $(i,j) \in b$ above ensures that for exactly one cell $(i,j)$ in the region $b$ we will have $x_{ijk} = 1$, i.e., digit $k$ in cell $(i,j)$.


- _Some cells must have the given digits_: $$ \forall (i,j) \in GD \;\; : \;\; \sum_{k} k*x_{ijk} = GD[i,j]$$ 
    
    $(i,j)\in GD$ are precisely

### Objective Function
There is no objective to maximize or minimize in this problem. We only need to find one infeasible solution (which turns out to be unique in this case). But there is no problem if we define an objective function like this:
$$\min z.$$

### Final Formulation
$$
\begin{eqnarray}
\begin{array}{rcl}
& \min & z\\
& \text{s.t.}& \sum_j x_{ijk} = 1, \forall i\in I, k\in K,\\
&& \sum_i x_{ijk} = 1, \forall j\in J, k\in K,\\
&& \sum_k x_{ijk} = 1, \forall i\in I, j\in J, \\
&& \sum_{(i, j) \in b}\sum_{k\in K} kx_{ijk} = z, \forall b\in B,\\
&& x_{ijk} \in \{0, 1\}, \forall i \in I, j\in J, k\in K,\\
&& z \geq 0.
\end{array}
\end{eqnarray}
$$